In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import fastText
import math
import numpy as np 
from numpy import random
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Conv1D, Conv2D, Dropout, MaxPooling1D, GlobalMaxPooling1D, Bidirectional, Input, Masking, Flatten, Concatenate
from keras import regularizers
import os
import re
import threading

/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [20]:
line_number=0
lock = threading.Lock()

In [4]:
os.getcwd()

'/home/jindal/notebooks/jindal/NER/language_model'

In [5]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [6]:
# words = ["mein", "dein", "unser", "mein"]
# for w in words:
#     if w in word_vectors:
#         print("I know " + w)
#     else:
#         wv = ft.get_word_vector(w)
#         print(wv)
#         word_vectors[w] = wv

In [7]:
print(nb_embedding_dims)

300


In [ ]:
print(ft.get_word_vector("mein"))

In [24]:
def my_generator(features, labels, batch_size):
    
    global line_number, lock
    batch_features = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims)) # initializing features with zeros
    batch_labels = np.zeros((batch_size, 2)) # 2 as one hot

    while True:
        # print(len(features))
        for i in range(batch_size):
            
            lock.acquire()
            index = line_number%len(features)
            line_number+=1
            lock.release()
#             index = random.choice(len(features), 1)[0]
#             print(index)
            batch_features[i] = process_features(features[index], nb_sequence_length, nb_embedding_dims)
            # print(batch_features[i])
            # print(batch_features[i].shape)
            batch_labels[i] = labels[index]
        yield batch_features, batch_labels

In [9]:
word_vectors = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims): # given a sentence, returns the embedding for it of fixed size
    words = re.compile('[\w-]+|[\W ]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    features = np.zeros((nb_sequence_length, nb_embedding_dims))
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors:
            wv = word_vectors[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors[w] = wv
        features[idx] = wv
        # print(str(idx) + " " + w)
        idx = idx + 1
    return features

In [10]:
train_lines = [line.strip().split("\t") for line in open('/home/jindal/notebooks/Resources/OffLang/sample_train.txt', encoding = "UTF-8")]
dev_lines = [line.strip().split("\t") for line in open('/home/jindal/notebooks/Resources/OffLang/sample_dev.txt', encoding = "UTF-8")]

In [11]:
from keras.utils import to_categorical

train_sentences = [x[0] for x in train_lines]
train_labels = to_categorical([0 if x[1] == "OTHER" else 1 for x in train_lines])
# train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]

dev_sentences = [x[0] for x in dev_lines]
dev_labels = to_categorical([0 if x[1] == "OTHER" else 1 for x in dev_lines])
# dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]

In [ ]:
# print(train_labels)

In [ ]:
model = Sequential([
    LSTM(64, recurrent_dropout = 0.5, dropout = 0.5, activation = 'relu', input_shape=(nb_sequence_length, nb_embedding_dims)),
    Dense(32, activation = 'relu'),
    Dropout(0.2),
    Dense(2, activation = 'softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])

In [ ]:
# model = Sequential([
#     Conv1D(128, kernel_size = 3, padding = 'valid', input_shape=(nb_sequence_length, nb_embedding_dims), activation = 'relu'),
#     MaxPooling1D(5),
#     Flatten(),
#     Dense(64, activation = 'relu'),
#     Dropout(0.2),
#     Dense(2, activation = 'softmax')
# ])
# model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])

In [12]:
filter_sizes = (3, 4, 5)
model_input = Input(shape = (nb_sequence_length, nb_embedding_dims))
model_layers = Dropout(0.8)(model_input)
conv_blocks = []
for sz in filter_sizes:
    conv = Conv1D(filters = 100,
                         kernel_size = sz,
                         padding = "valid",
                         activation = "relu",
                         strides = 1,
                 kernel_regularizer = regularizers.l2(0.0001))(model_layers)
    conv = GlobalMaxPooling1D()(conv)
    # conv = Flatten()(conv)
    conv_blocks.append(conv)
model_concatenated = Concatenate()(conv_blocks)
model_concatenated = Dropout(0.8)(model_concatenated)
model_concatenated = Dense(64, activation = "relu")(model_concatenated)
model_output = Dense(2, activation = "softmax")(model_concatenated)
model = Model(model_input, model_output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [13]:
samples_per_epoch = len(train_sentences)
epochs = 50
batch_size = 32
steps_per_epoch = math.ceil(samples_per_epoch / batch_size)

In [25]:
model.fit_generator(
    my_generator(train_sentences, train_labels, batch_size), 
    steps_per_epoch=steps_per_epoch, nb_epoch=epochs,
    validation_data = my_generator(dev_sentences, dev_labels, batch_size),
    validation_steps = math.ceil(len(dev_sentences) / batch_size)
)

Epoch 1/50
 10/132 [=>............................] - ETA: 1s - loss: 0.6908 - acc: 0.6281

/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """
/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=132, validation_steps=26, validation_data=<generator..., epochs=50)`
  """


132/132 [==============================] - 1s 11ms/step - loss: 0.7128 - acc: 0.6423 - val_loss: 0.7287 - val_acc: 0.6394
Epoch 2/50
132/132 [==============================] - 2s 11ms/step - loss: 0.6942 - acc: 0.6671 - val_loss: 0.7049 - val_acc: 0.6803
Epoch 3/50
132/132 [==============================] - 2s 12ms/step - loss: 0.6914 - acc: 0.6624 - val_loss: 0.7035 - val_acc: 0.6791
Epoch 4/50
132/132 [==============================] - 2s 12ms/step - loss: 0.6861 - acc: 0.6681 - val_loss: 0.6996 - val_acc: 0.6779
Epoch 5/50
132/132 [==============================] - 2s 12ms/step - loss: 0.6979 - acc: 0.6494 - val_loss: 0.7017 - val_acc: 0.6611
Epoch 6/50
132/132 [==============================] - 2s 12ms/step - loss: 0.6881 - acc: 0.6648 - val_loss: 0.6962 - val_acc: 0.6719
Epoch 7/50
132/132 [==============================] - 2s 12ms/step - loss: 0.6840 - acc: 0.6626 - val_loss: 0.6855 - val_acc: 0.6767
Epoch 8/50
132/132 [==============================] - 2s 12ms/step - loss: 0.689